# A test run of STHD on a sample patch, and visualize per spot predicted cell type STHD_pred_ct and cell type probabilities

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import numba
import squidpy as sq
from numba import jit, njit, prange

import pandas as pd
import matplotlib.pyplot as plt
import squidpy as sq
import sys
sys.path.append('../STHD')
import model
import refscrna
import sthdio
import train
import qcmask
import color_palette
import config


/hpc/group/yizhanglab/yiz/venv/stvit-cpu/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


## Take a patch cropped in s02

In [2]:
patch_path = '../analysis/20240527_update_sthd/crop10_large/'
refile = '/hpc/home/yz922/yizhanglab/yz922/proj/STHD/STHD_data/colon_cancer_celltype_average_expr_genenorm_rctd_style_0525_0.000125_log2.0.5_4618gs.txt'
# preset parameters
n_iter =  20
step_size = 1
beta = 0.1

## Train STHD on this patch - take a minute. Models are saved in the same patch path.
### (Optional: qc mask to filter low counts; or provide a separate mask)

In [ ]:
sthdata = train.load_data(patch_path)
print(sthdata.adata.shape)
sthdata.adata = qcmask.background_detector(sthdata.adata, threshold = 50, n_neighs =4, n_rings = 2)
print(sthdata.adata.shape)
qcmask.visualize_background(sthdata)

In [4]:
sthdata = train.load_data(patch_path)
print(sthdata.adata.shape)
sthdata.adata = qcmask.background_detector(sthdata.adata, threshold = 50, n_neighs =4, n_rings = 2)
print(sthdata.adata.shape)
#qcmask.visualize_background(sthdata)
sthdata_filtered = qcmask.filter_background(sthdata, threshold = 50 )
print(sthdata.adata.shape)
sthdata_filtered, genemeanpd_filtered = train.sthdata_match_refgene(sthdata_filtered, refile)
P_filtered = train.train(sthdata_filtered, n_iter, step_size, beta)
P = train.fill_p_filtered_to_p_full(P_filtered, sthdata_filtered, genemeanpd_filtered, sthdata )
sthdata = train.predict(sthdata, P, genemeanpd_filtered, mapcut= 0.8)
pdata = train.save_prediction_pdata(sthdata, file_path = patch_path)

/hpc/group/yizhanglab/yiz/venv/stvit-cpu/lib64/python3.9/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


[log] Number of spots:  278766
(278766, 18085)
(278766, 18085)
[Log] filtering background: 278766 spots to 264081 spots
(278766, 18085)


/hpc/group/yizhanglab/yiz/venv/stvit-cpu/lib64/python3.9/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


[Log] num of gene overlap 4212
[Log] attaching ref gene expr in .lambda_cell_type_by_gene_matrix
cut 18085 genes to match to reference 4212 genes
[Log] prepare_constants and training weights
Currently we only support symmetric adjacency matrix of neighbors
[Log] Training...
0 113.31753124601151 -107.86132299746838 54.56208248543135
1 107.78209306604275 -102.26194464794528 55.20148418097461
2 103.53147691833999 -98.17562495208468 53.558519662553095
3 100.49468093684494 -95.43582428131702 50.5885665552792
4 98.1208400091895 -93.38897341737444 47.31866591815054
5 96.10403071335567 -91.68265448383504 44.21376229520638
6 94.68874529286512 -90.5553296418223 41.33415651042825
7 93.70038860627604 -89.84121251112633 38.59176095149702
8 92.90269760507904 -89.3174434771086 35.85254127970449
9 92.19812319370219 -88.8938337612761 33.04289432426083
10 91.55874714839285 -88.54100652393419 30.177406244586656
11 91.00813211369764 -88.27264867005488 27.35483443642768
12 90.57670359871778 -88.10713633315

/hpc/group/yizhanglab/yiz/venv/stvit-cpu/lib64/python3.9/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/hpc/group/yizhanglab/yiz/projects/sthd_study/STHD/STHD_nb_update/../STHD/train.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adata.obs['p_ct_'+ct] = p[:,i]
/hpc/group/yizhanglab/yiz/projects/sthd_study/STHD/STHD_nb_update/../STHD/train.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ad

[Log]Predicted cell type in STHD_pred_ct in adata.obs
[Log]Predicted cell type probabilities in columns starting with p_ct_ in adata.obs
[Log] prediction saved to ../analysis/20240527_update_sthd/crop10_large/_pdata.tsv


In [5]:
'''
### Or, train simply without masking any region.

sthdata = load_data(patch_path)
sthdata, genemeanpd_filtered = sthdata_match_refgene(sthdata, refile, matchrefgene=True, ref_renorm = False)
P = train.train(sthdata, n_iter, step_size, beta)
sthdata = train.predict(sthdata, P, genemeanpd_filtered, mapcut= 0.8) 
pdata = train.save_prediction_pdata(sthdata, file_path = patch_path, prefix = '')
'''

"\n### Or, train simply without masking any region.\n\nsthdata = load_data(patch_path)\nsthdata, genemeanpd_filtered = sthdata_match_refgene(sthdata, refile, matchrefgene=True, ref_renorm = False)\nP = train.train(sthdata, n_iter, step_size, beta)\nsthdata = train.predict(sthdata, P, genemeanpd_filtered, mapcut= 0.8) \npdata = train.save_prediction_pdata(sthdata, file_path = patch_path, prefix = '')\n"

In [ ]:
cmap = color_palette.get_config_colormap( name = 'colormap_coloncatlas_98')
data_palette = color_palette.prepare_palette(cmap, sthdata.adata)

sq.pl.spatial_scatter(sthdata.adata, 
                      color='STHD_pred_ct', 
                      crop_coord = [sthdata.get_sequencing_data_region()],
                      legend_fontsize=8,
                      palette = data_palette,
                      figsize=(12,12)
                    
                     )
#plt.savefig('../analysis/figure/fig1c_crop10_pred_ct.pdf',dpi=300)

### or, load model results

In [7]:
sthdata = train.load_data_with_pdata(file_path = patch_path)

/hpc/group/yizhanglab/yiz/venv/stvit-cpu/lib64/python3.9/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


[log] Number of spots:  278766
[Log] Loading prediction into sthdata.adata.obs, overwriting
Index([], dtype='object')


### some simple plotting to visualize results 

In [8]:
# -----------------------Quick Visualization----------------------------
def analysis(sthd_data, color_list1, color_list2):
    adata = sthd_data.adata.copy()
    crop_coor = sthdata.get_sequencing_data_region()
    
    f, ax = plt.subplots(2, 3, figsize=(40, 20)) # row, col
    
    cur_row = 0
    for i, c in enumerate(color_list1):
        sq.pl.spatial_scatter(adata, color=c, crop_coord = [crop_coor], ax=ax[cur_row, i], vmin=0) #[]todo. we put p=-1 for the filtered spots
        
    cur_row = 1
    for i, c in enumerate(color_list2):
        sq.pl.spatial_scatter(adata, color=c, crop_coord = [crop_coor], ax=ax[cur_row, i], vmin=0)
    return(adata)

In [ ]:
## coloring a few cell types's probablity
color_list1 = ['p_ct_Tumor cE03 (Stem/TA-like prolif)','p_ct_Tumor cE05 (Enterocyte 2)', 'p_ct_Tumor cE01 (Stem/TA-like)']
color_list2 = ['p_ct_cP2 (Plasma IgG)', 'p_ct_cM02 (Macrophage-like)', 'STHD_pred_ct', ]
adata = analysis(sthdata, color_list1, color_list2)


In [ ]:
crop_coor = sthdata.get_sequencing_data_region()

In [ ]:
#fig, ax = plt.subplots( figsize=(6,6))
sq.pl.spatial_scatter(adata, color=color_list1[0], crop_coord = [crop_coor],  vmin=0)
plt.savefig('../analysis/figure/crop10_prob_cE03.pdf',dpi=300)

In [ ]:
#fig, ax = plt.subplots( figsize=(6,6))
sq.pl.spatial_scatter(adata, color=color_list1[1], crop_coord = [crop_coor],  vmin=0)
plt.savefig('../analysis/figure/crop10_prob_cE05.pdf',dpi=300)

In [ ]:
#fig, ax = plt.subplots( figsize=(6,6))
sq.pl.spatial_scatter(adata, color=color_list1[2], crop_coord = [crop_coor],  vmin=0)
plt.savefig('../analysis/figure/crop10_prob_cE01.pdf',dpi=300)

In [ ]:
#fig, ax = plt.subplots( figsize=(6,6))
sq.pl.spatial_scatter(adata, color=color_list2[0], crop_coord = [crop_coor],  vmin=0)
plt.savefig('../analysis/figure/crop10_prob_plasma.pdf',dpi=300)

In [ ]:
#fig, ax = plt.subplots( figsize=(6,6))
sq.pl.spatial_scatter(adata, color=color_list2[1], crop_coord = [crop_coor],  vmin=0)
plt.savefig('../analysis/figure/crop10_prob_macro.pdf',dpi=300)